In [10]:
from google.cloud import bigquery
import pandas as pd

# Initialize client
client = bigquery.Client(project="students-group2")

# 1. Check the movies table structure
query_movies = """
SELECT *
FROM `master-ai-cloud.MoviePlatform.movies`
LIMIT 5
"""

df_movies = client.query(query_movies).to_dataframe()
print("Movies Table Sample:")
print(df_movies.head())
print("\nMovies Columns:", df_movies.columns.tolist())

Movies Table Sample:
   movieId                             title              genres
0   126929              Li'l Quinquin (    )  (no genres listed)
1   135460                      Pablo (2012)  (no genres listed)
2   138863  The Big Broadcast of 1936 (1935)  (no genres listed)
3   141305       Round Trip to Heaven (1992)  (no genres listed)
4   141472       The 50 Year Argument (2014)  (no genres listed)

Movies Columns: ['movieId', 'title', 'genres']


In [3]:
# 2. Check the ratings table structure
query_ratings = """
SELECT *
FROM `master-ai-cloud.MoviePlatform.ratings`
LIMIT 5
"""

df_ratings = client.query(query_ratings).to_dataframe()
print("\nRatings Table Sample:")
print(df_ratings.head())
print("\nRatings Columns:", df_ratings.columns.tolist())


Ratings Table Sample:
   userId  movieId  rating   timestamp
0       1      204     0.5  1217895786
1       1      256     0.5  1217895764
2       1      277     0.5  1217895772
3       1      719     0.5  1217895799
4       1    45950     0.5  1217897813

Ratings Columns: ['userId', 'movieId', 'rating', 'timestamp']


In [4]:
# 3. Get basic statistics (CAREFUL - this processes more data)
query_stats = """
SELECT 
    COUNT(DISTINCT userId) as num_users,
    COUNT(DISTINCT movieId) as num_movies,
    COUNT(*) as num_ratings,
    AVG(rating) as avg_rating,
    MIN(rating) as min_rating,
    MAX(rating) as max_rating
FROM `master-ai-cloud.MoviePlatform.ratings`
"""

df_stats = client.query(query_stats).to_dataframe()
print("\nDataset Statistics:")
print(df_stats)


Dataset Statistics:
   num_users  num_movies  num_ratings  avg_rating  min_rating  max_rating
0        668       10325       105339     3.51685         0.5         5.0


In [11]:
# 5. Check user activity distribution
query_user_activity = """
SELECT 
    userId,
    COUNT(*) as num_ratings
FROM `master-ai-cloud.MoviePlatform.ratings`
GROUP BY userId
ORDER BY num_ratings DESC
LIMIT 20
"""

df_user_activity = client.query(query_user_activity).to_dataframe()
print("\nTop 20 Most Active Users:")
print(df_user_activity)
print(f"\nAverage ratings per user: {105339/668:.1f}")


Top 20 Most Active Users:
    userId  num_ratings
0      668         5678
1      575         2837
2      458         2086
3      232         1421
4      310         1287
5      475         1249
6      128         1231
7      224         1182
8      607         1176
9       63         1107
10     451         1064
11     627         1050
12     413         1040
13      62         1032
14     461         1005
15     164          895
16     354          887
17     402          865
18     220          834
19     109          804

Average ratings per user: 157.7


In [12]:
# 6. Check movie popularity
query_movie_popularity = """
SELECT 
    m.movieId,
    m.title,
    COUNT(*) as num_ratings,
    AVG(r.rating) as avg_rating
FROM `master-ai-cloud.MoviePlatform.ratings` r
JOIN `master-ai-cloud.MoviePlatform.movies` m
ON r.movieId = m.movieId
GROUP BY m.movieId, m.title
ORDER BY num_ratings DESC
LIMIT 20
"""

df_popular = client.query(query_movie_popularity).to_dataframe()
print("\nTop 20 Most Rated Movies:")
print(df_popular)


Top 20 Most Rated Movies:
    movieId                                              title  num_ratings  \
0       296                                Pulp Fiction (1994)          325   
1       356                                Forrest Gump (1994)          311   
2       318                   Shawshank Redemption, The (1994)          308   
3       480                               Jurassic Park (1993)          294   
4       593                   Silence of the Lambs, The (1991)          290   
5       260          Star Wars: Episode IV - A New Hope (1977)          273   
6      2571                                 Matrix, The (1999)          261   
7       589                  Terminator 2: Judgment Day (1991)          253   
8       527                            Schindler's List (1993)          248   
9       110                                  Braveheart (1995)          248   
10      457                               Fugitive, The (1993)          244   
11        1              

In [13]:
# 7. Check genre distribution (excluding "no genres listed")
query_genres = """
SELECT 
    genres,
    COUNT(*) as num_movies
FROM `master-ai-cloud.MoviePlatform.movies`
WHERE genres != '(no genres listed)'
GROUP BY genres
ORDER BY num_movies DESC
LIMIT 20
"""

df_genres = client.query(query_genres).to_dataframe()
print("\nTop 20 Genre Combinations:")
print(df_genres)


Top 20 Genre Combinations:
                         genres  num_movies
0                         Drama        1385
1                        Comedy         826
2                  Comedy|Drama         465
3                 Drama|Romance         421
4                Comedy|Romance         363
5                   Documentary         300
6          Comedy|Drama|Romance         286
7                Drama|Thriller         212
8                   Crime|Drama         188
9                        Horror         170
10         Crime|Drama|Thriller         163
11              Horror|Thriller         152
12                    Drama|War         146
13                     Thriller         106
14                 Comedy|Crime          92
15                Action|Comedy          77
16        Action|Crime|Thriller          75
17  Action|Crime|Drama|Thriller          69
18              Action|Thriller          69
19                 Action|Drama          67


In [14]:
# 8. Find users with few ratings (cold start candidates)
query_cold_start = """
SELECT 
    COUNT(*) as num_users_with_few_ratings
FROM (
    SELECT userId, COUNT(*) as num_ratings
    FROM `master-ai-cloud.MoviePlatform.ratings`
    GROUP BY userId
    HAVING num_ratings <= 20
)
"""

df_cold_start = client.query(query_cold_start).to_dataframe()
print("\nUsers with 20 or fewer ratings:")
print(df_cold_start)


Users with 20 or fewer ratings:
   num_users_with_few_ratings
0                          19
